In [34]:
import spacy
nlp = spacy.load('en')
print 'ok. topic extraction can begin'

ok. topic extraction can begin


In [42]:
import glob
import itertools
import networkx


def browse_recursive(folder_path):
    fileList = glob.glob(folder_path+'/*')
    print 'no of files', str(len(fileList))
    errorlist = []
    for f in fileList:
        yield f

def get_file_content(file_path):
    f = open(file_path, 'r')
    content = f.read()
    f.close()
    return content

def parse_file_content(content):
    doc = nlp(unicode(content, "utf-8"))
    return doc

def get_named_entities(doc):
    entities3 = doc.ents
    for e in entities3:
        print e.text, e.label_ , [w.tag_ for w in e]

def get_candidate_keyphrases(doc):
#     nchunks = doc.noun_chunks
#     print 'Candidate Keyphrases'
#     for nc in nchunks:
#         print nc
    
    nounphrases = [np.orth_ for np in doc.noun_chunks]
    return nounphrases

def np_drop_singletermcommonnoun(noun_chunk):
    tokens = [(token.orth_, token.pos_) for token in noun_chunk]
    if(len(tokens) == 1 and tokens[0][1] == 'NOUN'):
        return []
    else:
        return tokens
    
def np_drop_unwanted(noun_chunk):
    unwanted = ['SPACE', 'DET', 'NUM']
    stop_words = set('for a of the and to in what my you this is your where'.split())
    tokens = [(token.orth_.lower(), token.pos_) for token in noun_chunk 
                  if token.pos_ not in unwanted and token.orth_ not in stop_words]
    return tokens

def np_drop_typicalcommonwords(noun_chunk):
    common_words = set('information meeting review'.split())
    tokens = [(token.orth_.lower(), token.pos_) for token in noun_chunk 
                  if token.orth_ not in common_words]
    return tokens
    
def create_graph():
    graph = networkx.Graph()
    return graph

def add_candidate_phrases_as_graph_edges(all_doc_phrases, graph):
    all_doc_phrases1 = iter(all_doc_phrases)
    all_doc_phrases2 = itertools.islice(all_doc_phrases1, 1, None)
    sequential_pairs = itertools.izip(all_doc_phrases1, all_doc_phrases2)
    for phrase1, phrase2 in sequential_pairs:
        graph.add_edge(phrase1, phrase2)

def save_graph(graph, output_file):
    networkx.write_gexf(graph, output_file )
    

def python_iter_test():
    t = [1,2,3,4,5]
    it1 = iter(t)
    it2 = iter(t)
    it2 = itertools.islice(it2, 1, None)
    
    
    
    pairs = itertools.izip(it1, it2)
    print list(pairs)
#     it = iter(t)
#     print list(it)
#     pairs = zip(t[::2], t[1::2])
#     print pairs

python_iter_test()
    
    
    

[(1, 2), (2, 3), (3, 4), (4, 5)]


In [43]:
def main():
    root_folder='/home/mprakash/Projects/enron-experiments/significant-terms/output/firstpass/maildir/delainey-d'
    graph_output_file_path = '/home/mprakash/Projects/enron-experiments/significant-terms/output/delainey-topic-graph.gexf'
    G = create_graph()
    for f in browse_recursive(root_folder):
        try:
            doc = parse_file_content(get_file_content(f))
            add_candidate_phrases_as_graph_edges(get_candidate_keyphrases(doc), G)
#             print 'success', f
        except Exception as ex:
            print 'error ', ex, 'for file ', f
    save_graph(G, graph_output_file_path)
    print 'topic graph created'

main()
        
    

no of files 910
error   for file  /home/mprakash/Projects/enron-experiments/significant-terms/output/firstpass/maildir/delainey-d/maildir_delainey-d_all_documents_883.
error   for file  /home/mprakash/Projects/enron-experiments/significant-terms/output/firstpass/maildir/delainey-d/maildir_delainey-d_all_documents_908.
topic graph created
